# **Tratamento** para estimação do potencial de trabalho remoto no Brasil em 2019 

A estimação será feita usando, primeiramente, a base da PNAD Contínua suplementar de 2019 e, depois, a PNAD COVID

Foi adotado a hipótese de que a perda de potencial de trabalho remoto devido à falta de infraestrutura é a mesma na PNAD-COVID que na PNAD Contínua de 2019.

Trabalhar com as seguintes variáveis da PNAD Contínua: 
- V4010, que classifica as diversas ocupações, 
- a unidade da federação (UF), 
- gênero (V2007), 
- raça (V2010), 
- renda em todos os trabalhos (VD4019), 
- Setor Público (V4013, Código da principal atividade desse negócio/empresa),
- educação (VD3004), 
- formal/informal (combinação da variável VD4009 com a variável V4019) e 
- o peso amostral (V1032).

Da mesma forma, os equivalentes para a PNAD-COVID são:
- V1013, mês da pesquisa,
- V1012, semana da pesquisa,
- UF, unidade da federação,
- C007C, trabalho, cargo ou ocupação,
- A003, gênero,
- A004, raça,
- A005, escolaridade,
- C007D, setor público,
- C01012, renda, 
- C007, formal informal e
- o peso amostral, V1032.

Não esquecer de filtrar a base para os registros que a 'dummy_infra' igual à 1.

In [22]:
#pacotes 
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#Importação dos dados da PNAD Contínua 2019 (suplementar)

#determina qual diretório usar para cada sistema operacional.
# import os
# if os.name == 'nt': #windows
    # diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\suplementar\\'
# 
# elif: #linux(ou mac)
    # diretorio = '/home/matheus/Microdados PNAD/feather/'
# 

filtro_pnadc = ['Trimestre','V4010', 'UF', 'V2007', 'V2010', 'VD4019', 'V4013', 'VD3004', 'VD4009', 'V4019', 'V1032', 'dummy_infra']
pnadc_19 = pd.read_parquet('../Dados/pnad/PNADC_2019_editado.parquet', columns=filtro_pnadc)

In [8]:
#Importação dos códigos de ocupação traduzidos

ocupacoes = pd.read_excel('../Anexos/ocupacoes_traduzidas.xlsx')

In [23]:
#Importação dos dados da PNAD COVID

#determina qual diretório usar para cada sistema operacional.
# import os
# if os.name == 'nt': #windows
#     diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\feather_covid\\'

# else: #linux(ou mac)
#     diretorio = '/home/matheus/Microdados PNAD/PNAD_COVID/'

# #Leitura de todas as bases com as variáveis selecionadas e criação de variáveis dinamicamente.

filtro_pnad_covid = ['Ano', 'V1013', 'V1012', 'UF', 'C007C', 'A003', 'A004', 'A005', 'C007D', 'C01012', 'C007', 'V1032']

diretorio = '../Dados/pnad_covid/'

for i in range(5, 12):
    if i < 10:
        mes = '0' + str(i)
    else:
        mes = i
    print(f'Importando PNAD COVID {mes} 2020')
    globals()[f'pnad_covid_{mes}2020'] = pd.read_parquet(fr'{diretorio}PNAD_COVID_{mes}2020.parquet', columns=filtro_pnad_covid)

print('Fim da importação')

Importando PNAD COVID 05 2020
Importando PNAD COVID 06 2020
Importando PNAD COVID 07 2020
Importando PNAD COVID 08 2020
Importando PNAD COVID 09 2020
Importando PNAD COVID 10 2020
Importando PNAD COVID 11 2020
Fim da importação


### Junção dos códigos de ocupação com os dados da PNAD Contínua 2019

In [5]:
#Coluna com o código da ocupação Major das ocupações traduzidas
pnadc_19['COD_OCUP_MAJ'] = pnadc_19['V4010'].astype(str).str[:1]

In [6]:
ocupacoes['Cod'] = ocupacoes['Cod'].astype(str)

In [7]:
pnadc_19_remoto = pd.merge(pnadc_19, ocupacoes, left_on='COD_OCUP_MAJ', right_on='Cod', how='left')

In [8]:
pnadc_19_remoto.head(2)

,Trimestre,V4010,UF,V2007,V2010,VD4019,V4013,VD3004,VD4009,V4019,V1032,dummy_infra,COD_OCUP_MAJ,code_ONET,Title_ONET,teleworkable,Cod,Titulação
0,1,3221,Rondônia,Mulher,Parda,1583.0,86001,Superior completo,Empregado no setor privado com carteira de tra...,NaN,149.43857,0,3,11-9141,"Property, Real Estate, and Community Associati...",1.0,3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO
1,1,3221,Rondônia,Mulher,Parda,1583.0,86001,Superior completo,Empregado no setor privado com carteira de tra...,NaN,149.43857,0,3,13-1021,"Buyers and Purchasing Agents, Farm Products",1.0,3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO


Até aqui, a PNAD Contínua com a dummy de infraestrutura e a base de ocupações traduzidas da O*NET para COD foram e unidas no dataframe <code>pnadc_19_remoto</code>.

### Classificação das ocupações na PNAD COVID e junção com a base de ocupações traduzidas da O*NET para COD

In [17]:
dic_ocupacao_subgrupo = {"Engenheiros e profissionais das ciências exatas e naturais (nível superior)" : 21,
"Médicos, Enfermeiros e Profissionais de Saúde (nível superior)" : 22,
"Professores, pedagogos" : 23,
"profissionais de administração e negócios (nível superior)" : 24,
"profissionais de tecnologia da informação e comunicação (nível superior)" : 25,
"Profissionais de nível superior do direito e ciências sociais; religiosos e artistas" : 26,
"Técnicos e profissionais de nível médio dadas ciências exatas e naturais e da Engenharia" : 31,
"Técnicos e profissionais de nível médio da saúde " : 32,
"Técnicos e profissionais de nível médio da administração, contabilidade e negócios" : 33,
"Técnicos e profissionais de nível médio do direito, Ciências Sociais, Atividades Artísticas e Culturais e Afins" : 34,
"Técnicos e profissionais de nível médio de Informática e Comunicação" : 35,
"Auxiliares administrativos, secretários de escritório" : 41,
"Recepcionistas, Atendentes de telemarketing e outros Funcionários de atendimento ao cliente" : 42,
"Trabalhadores de contabilidade, controle de estoque, material" : 43,
"Outros trabalhadores de apoio administrativo" : 44,
"Cozinheiros, garçons, cabelereiros, porteiros e outros trabalhadores dos serviços pessoais" : 51,
"Comerciantes e vendedores" : 52,
"Cuidadores de crianças, doentes e idosos" : 53,
"Seguranças, vigilantes, policiais civis e outros Trabalhadores dos serviços de proteção" : 54,
"Agricultores, criadores de animais, pescadores, silvicultores e jardineiros" : 60,
"Pedreiros, pintores, eletricistas, mecânicos, marceneiros, artesãos, costureiros, padeiros " : 70,
"Operadores de máquinas e montadores indústria; condutores de veículos" : 80,
"Empregados domésticos, auxiliares de limpeza, auxiliares de produção, carregadores, vendedores ambulantes e outras ocupações elementares na agropecuária, indústria, comércio e serviços" : 90,}

In [73]:
#Dicionário com ocupaçoes da pnad covid classificada pelo maior grupo da de ocupação da COD
dict_ocupacoes_pnad_covid = {
    'Técnico, profissional da saúde de nível médio':32,
    'Secretária, recepcionista':42,
    'Vendedor ambulante (feirante, camelô, comerciante de rua, quiosque)':90,
    'Professor da educação infantil, de ensino fundamental, médio ou superior,':23,
    'Pedreiro, servente de pedreiro, pintor, eletricista, marceneiro':70,
    'Outros':0, 
    'Porteiro, zelador':51, 
    'Padeiro, açougueiro e doceiro':70,
    'Motorista de caminhão (caminhoneiro),':80,
    'Artesão, costureiro e sapateiro':70,
    'Outra profissão de nível superior (advogado, engenheiro, contador, jornalista etc.)':25,
    'Segurança, vigilante, outro trabalhador dos serviços de proteção':54,
    'Faxineiro, auxiliar de limpeza etc. (em empresa pública ou privada),':90,
    'Diretor, gerente, cargo político ou comissionado':24,
    'Balconista, vendedor de loja':52,
    'Operador de máquinas, montador na indústria;':80,
    'Médico, enfermeiro, profissionais de saúde de nível superior':22,
    'Cabeleireiro, manicure e afins':51,
    'Auxiliar de escritório, escriturário':41,
    'Mecânico de veículos, máquinas industriais etc.':70,
    'Motorista (de aplicativo, de taxi, de van, de mototáxi, de ônibus)':80,
    'Cozinheiro e garçom (de restaurantes, empresas)':51,
    'Comerciante (dono do bar, da loja etc.)':52, 
    'Policial civil':54,
    'Empregado doméstico, diarista, cozinheiro (em domicílios particulares),':90,
    'Vendedor a domicílio, representante de vendas, vendedor de catálogo (Avon, Natura etc.)':52,
    'Outro técnico ou profissional de nível médio':34,
    'Agricultor, criador de animais, pescador, silvicultor e jardineiro':60,
    'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Eats, IFood, Rappy etc.)':90,
    'Auxiliar de produção, de carga e descarga;':90,
    'Pedagogo, professor de idiomas, música, arte e reforço escolar':23,
    'Motoboy':9,
    'Artista, religioso (padre, pastor etc.)':26,
    'Operador de Telemarketing':42,
    'Auxiliar da agropecuária (colhedor de frutas, boia fria, etc.)':90,
    'Cuidador de crianças, doentes ou idosos':53,
    'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Esta, Iodo, Rapa etc.)':90
    }

In [74]:
for i in range(5, 12):
    if i < 10:
        mes = '0' + str(i)
    else:
        mes = i
    print(f'Classificando PNAD COVID {mes} 2020')
    globals()[f'pnad_covid_{mes}2020']['COD_OCUP_MAJ'] = globals()[f'pnad_covid_{mes}2020']['C007C'].map(dict_ocupacoes_pnad_covid)
    globals()[f'pnad_covid_{mes}2020']['COD_OCUP_MAJ'].fillna(999, inplace=True)
    globals()[f'pnad_covid_{mes}2020']['COD_OCUP_MAJ'] = globals()[f'pnad_covid_{mes}2020']['COD_OCUP_MAJ'].astype(int)
print('Fim da classificação')

Classificando PNAD COVID 05 2020
Classificando PNAD COVID 06 2020
Classificando PNAD COVID 07 2020
Classificando PNAD COVID 08 2020
Classificando PNAD COVID 09 2020
Classificando PNAD COVID 10 2020
Classificando PNAD COVID 11 2020
Fim da classificação


In [5]:
pnad_covid = pd.concat([pnad_covid_052020, pnad_covid_062020, pnad_covid_072020, pnad_covid_082020, pnad_covid_092020, pnad_covid_102020, pnad_covid_112020], ignore_index=True)

In [41]:
subgrupo_filtrado = ocupacoes[(ocupacoes['Subgrupos Principal'] > 10)&(ocupacoes['Subgrupos Principal'] < 100)].drop_duplicates(subset=['Subgrupos Principal'], keep='first')

In [53]:
subgrupo_filtrado = subgrupo_filtrado[['Grandes Grupos', 'Subgrupos Principal', 'teleworkable']]

In [10]:
ocupacoes = ocupacoes[['Cod','teleworkable','Titulação']]

In [60]:
#susbrecer valores de subgrupo principal maiores que 90 como 90

subgrupo_filtrado['Subgrupos Principal'] = subgrupo_filtrado['Subgrupos Principal'].apply(lambda x: 90 if x > 90 else x)

/tmp/ipykernel_14086/1236885171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subgrupo_filtrado['Subgrupos Principal'] = subgrupo_filtrado['Subgrupos Principal'].apply(lambda x: 90 if x > 90 else x)


In [65]:
subgrupo_filtrado.drop_duplicates(subset=['Subgrupos Principal'], keep='first', inplace=True)

/tmp/ipykernel_14086/526156553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subgrupo_filtrado.drop_duplicates(subset=['Subgrupos Principal'], keep='first', inplace=True)


In [25]:
# #Nova coluna com o código da ocupação Major das ocupações traduzidas
# pnad_covid['COD_OCUP_MAJ'] = pnad_covid['C007C'].map(dict_ocupacoes_pnad_covid)
# #converter para inteiro
# pnad_covid['COD_OCUP_MAJ'].fillna(999, inplace=True)
# pnad_covid['COD_OCUP_MAJ'] = pnad_covid['COD_OCUP_MAJ'].astype(int)


In [11]:
ocupacoes['Cod'] = ocupacoes['Cod'].astype(int)

In [76]:
for i in range(5, 12):
    if i < 10:
        mes = '0' + str(i)
    else:
        mes = i
    globals()[f'pnad_covid_{mes}2020'] = pd.merge(globals()[f'pnad_covid_{mes}2020'], subgrupo_filtrado, left_on='COD_OCUP_MAJ', right_on='Subgrupos Principal', how='left')
    print(f'Merge de ocupações com PNAD COVID {mes} 2020')
    globals()[f'pnad_covid_{mes}2020'].to_parquet(f'../Dados/Classificado/pnad_covid_{mes}2020_classificado.parquet')
    print(f'arquivo parquet pnad_covid_{mes} criado')
    del globals()[f'pnad_covid_{mes}2020']
    print(f'Variável pnad_covid_{mes} deletada')
print('Fim do merge')

KeyError: 'pnad_covid_052020'

In [27]:
pnad_covid_remoto = pd.merge(pnad_covid, ocupacoes, left_on='COD_OCUP_MAJ', right_on='Cod', how='left')

In [ ]:
#Criação dos arquivos .feather para cada mês da PNAD COVID classificados
for i in range(5, 12):
    if i < 10:
        mes = '0' + str(i)
    else:
        mes = i
    print(f'Criando PNAD COVID {mes} 2020 classificada')
    globals()[f'pnad_covid_{mes}2020'].to_feather(f'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\suplementar\\pnad_covid_{mes}2020_classificada.feather')
print('Fim da criação')

In [ ]:
#apenas o valor agregado
pot_pnad_covid = pd.read_excel('..\\Anexos\\pot_pnad_covid.xlsx')
pot_pnad_infra = pd.read_excel('..\\Anexos\\pot_pnad_infra.xlsx')
pot_pnad = pd.read_excel('..\\Anexos\\pot_pnad.xlsx')

### Classificação das ocupações na PNAD COVID

In [29]:
classificao_pnad_covid = pd.read_excel('../Anexos/covid_ocupacoes_teleworkable.xlsx')

In [13]:
classificao_pnad_covid['teleworkable'] = classificao_pnad_covid['teleworkable'].astype(int)

### Classificação da pnad_covid

In [30]:
#criar dicionario com as Ocupações e o valor de teleworkable

dict_ocupacoes_pnad_covid = dict(zip(classificao_pnad_covid['Ocupação'], classificao_pnad_covid['Grupos de Base']))

In [32]:
classificacao_filtrado = classificao_pnad_covid[['Grandes Grupos','Grupos de Base','teleworkable']]

In [33]:
for i in range(5, 12):
    if i < 10:
        mes = '0' + str(i)
    else:
        mes = i
    globals()[f'pnad_covid_{mes}2020']['C007C'] = globals()[f'pnad_covid_{mes}2020']['C007C'].map(dict_ocupacoes_pnad_covid)
    globals()[f'pnad_covid_{mes}2020'] = pd.merge(globals()[f'pnad_covid_{mes}2020'], classificacao_filtrado, how='left', left_on='C007C', right_on='Grupos de Base')
    print(f'Merge de ocupações com PNAD COVID {mes} 2020')
    globals()[f'pnad_covid_{mes}2020'].to_parquet(f'../Dados/Classificado/pnad_covid_{mes}2020_classificado.parquet')
    print(f'arquivo parquet pnad_covid_{mes} criado')
    del globals()[f'pnad_covid_{mes}2020']
    print(f'Variável pnad_covid_{mes} deletada')
print('Fim do merge')

Merge de ocupações com PNAD COVID 05 2020
arquivo parquet pnad_covid_05 criado
Variável pnad_covid_05 deletada
Merge de ocupações com PNAD COVID 06 2020
arquivo parquet pnad_covid_06 criado
Variável pnad_covid_06 deletada
Merge de ocupações com PNAD COVID 07 2020
arquivo parquet pnad_covid_07 criado
Variável pnad_covid_07 deletada
Merge de ocupações com PNAD COVID 08 2020
arquivo parquet pnad_covid_08 criado
Variável pnad_covid_08 deletada
Merge de ocupações com PNAD COVID 09 2020
arquivo parquet pnad_covid_09 criado
Variável pnad_covid_09 deletada
Merge de ocupações com PNAD COVID 10 2020
arquivo parquet pnad_covid_10 criado
Variável pnad_covid_10 deletada
Merge de ocupações com PNAD COVID 11 2020
arquivo parquet pnad_covid_11 criado
Variável pnad_covid_11 deletada
Fim do merge
